In [1]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
    
import pandas as pd
import numpy as np
import ast

import tasks.run_helper

In [2]:
df = pd.read_csv(
    "../data/100_annotators.csv",
    converters={"annot_personality_characteristics": ast.literal_eval},
)
df["toxicity"] = df.annotation.apply(lambda x: x[-1]).astype(int)
df["annot_politics"] = df.annot_personality_characteristics.apply(
    lambda x: x[0]
)
df.annot_age = pd.cut(df.annot_age, bins=4)
df.message_id = df.message_id.astype(str)
df["comment_key"] = df.conv_id + df.message_id

df = df.loc[
    :,
    [
        "conv_id",
        "message_id",
        "comment_key",
        "message",
        "toxicity",
        "annot_age",
        "annot_sex",
        "annot_sexual_orientation",
        "annot_demographic_group",
        "annot_current_employment",
        "annot_education_level",
        "annot_politics",
    ],
]
df = df.groupby(["conv_id", "message_id", "comment_key", "message"]).apply(
    lambda x: pd.Series(
        {
            col: x[col].tolist()
            for col in df.columns
            if col not in ["conv_id", "message_id", "comment_key", "message"]
        }
    ),
    include_groups=False,
)
df["fake_index"] = 1
df["random"] = df.annot_politics.apply(
    lambda x: [np.random.randint(1, 10) for _ in range(len(x))]
)

In [3]:
sdb_columns = [
    "annot_age",
    "annot_sex",
    "annot_demographic_group",
    "annot_sexual_orientation",
    "annot_current_employment",
    "annot_education_level",
    "annot_politics"
]
res = tasks.run_helper.run_all_results(
    df=df,
    sdb_columns=sdb_columns,
    discussion_id_col="fake_index",
    value_col="toxicity",
    comment_key_col="comment_key",
)
res

Evaluating SDB dimensions:   0%|          | 0/7 [00:00<?, ?it/s]

1          
                                                     kappa    pvalue
sdb_column                                                          
annot_age                (14.927, 33.25]          0.171722  0.816667
                         (33.25, 51.5]            0.311972  0.816667
                         (51.5, 69.75]           -0.548673  0.816667
                         (69.75, 88.0]           -0.196136  0.816667
annot_current_employment blue-collar             -0.019307  0.800000
                         unemployed               0.631962  0.800000
                         white-collar            -0.414702  0.800000
annot_demographic_group  asian                    1.478304  0.816667
                         black                    0.509882  0.816667
                         other                   -2.156861  0.816667
                         white                    1.036186  0.816667
annot_education_level    high-school              0.938981  0.816667
                         none                    -1.749210  0.816667
                         university               0.243723  0.816667
annot_politics           apolitical              -2.665309  0.816667
                         left-wing liberal        1.056086  0.816667
                         right-wing conservative  0.671138  0.816667
annot_sex                female                  -0.155727  0.800000
                         male                    -0.487618  0.800000
                         non-binary               4.086664  0.800000
annot_sexual_orientation bisexual                 3.056395  0.816667
                         homosexual              -0.760057  0.816667
                         other                   -3.758392  0.816667
                         straight                 0.375954  0.816667

In [4]:
tasks.run_helper.run_result(
    df,
    discussion_id_col="fake_index",
    sdb_column="random",
    value_col="toxicity",
    comment_key_col="comment_key",
)

1          
      kappa    pvalue
6  1.761187  0.883333
8  2.390135  0.883333
3 -0.198528  0.883333
7  1.697698  0.883333
1  2.484884  0.883333
2 -1.778627  0.883333
4 -1.174539  0.883333
5 -0.454895  0.883333
9 -2.481735  0.883333